##### Version1<br>"democratic deliberation the illuminates different perspectives"

In [ ]:
# import os
# os.environ["GOOGLE_API_KEY"] = "<Your Gemini API>" # Put your Gemini API key here

question = "When and how will AGI surpass human intelligence?"

# 6 roles of evaluation:
role_prompts = {
    "Optimist": "You are the Optimist Evaluator. Provide 3 key benefits of the proposal.",
    "Pessimist": "You are the Pessimist Evaluator. Provide 3 key risks of the proposal.",
    "Conservative": "You are the Conservative Evaluator. Provide 3 key reasons to preserve tradition.",
    "Progressive": "You are the Progressive Evaluator. Provide 3 key innovative aspects.",
    "Authoritarian": "You are the Authoritarian Evaluator. Provide 3 key control mechanisms.",
    "Collectivist": "You are the Collectivist Evaluator. Provide 3 key communal benefits."
}


In [ ]:
from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, END
import google.generativeai as genai
from collections import Counter


class EvaluationState(TypedDict):
    proposal: Annotated[str, "Input: only one proposal per round"]
    loop_count: int
    evaluations: dict
    praetor_outputs: dict
    continue_: bool

In [ ]:
# making a node for each evaluation role

def make_gemini_node(role_name: str, prompt: str):
    model = genai.GenerativeModel("gemini-2.0-flash-lite")      # gemma-3n-e4b-it, gemma-3-27b-it
    def node_fn(state: EvaluationState):
        chat = model.start_chat(history=[])
        full_prompt = f"{prompt}\n\nProposal:\n{state['proposal']}"
        response = chat.send_message(full_prompt).text
        updated = dict(state)
        lc = str(state["loop_count"])
        if lc not in updated["evaluations"]:
            updated["evaluations"][lc] = {}
        updated["evaluations"][lc][role_name] = response
        return updated
    return node_fn

In [ ]:
# tge praetor node

def praetor_node(state: EvaluationState):
    loop_key = str(state["loop_count"])
    all_thoughts = []
    for output in state["evaluations"].get(loop_key, {}).values():
        thoughts = output.split("\n")
        all_thoughts.extend([t.strip() for t in thoughts if t.strip()])

    from collections import Counter
    counter = Counter(all_thoughts)
    top_3 = [t for t, _ in counter.most_common(3)]

    agreement = "High agreement among roles." if counter.most_common(1)[0][1] > 2 else "Low agreement. Roles are divergent."

    updated = dict(state)
    updated["praetor_outputs"][loop_key] = {
        "consensus": top_3,
        "summary": agreement
    }
    return updated

In [ ]:
# human interrupt node

def human_interrupt_node(state: EvaluationState):
    print(f"\n--- Praetor Summary (Loop {state['loop_count']}) ---")
    latest = state["praetor_outputs"][str(state["loop_count"])]
    for i, idea in enumerate(latest["consensus"], 1):
        print(f"{i}. {idea}")
    print("\n" + latest["summary"])

    decision = input("\nContinue to next loop? (y/n): ").lower().strip()
    updated = dict(state)
    updated["continue_"] = decision == "y"
    return updated

In [ ]:
def check_continue(state: EvaluationState) -> Literal["yes", "no"]:
    return "yes" if state["continue_"] else "no"

In [ ]:
# build the state graph

graph_builder = StateGraph(EvaluationState)

# Add dispatch and evaluator nodes
def dispatch_fn(state): return state
graph_builder.add_node("Dispatch", dispatch_fn)

for role, prompt in role_prompts.items():
    graph_builder.add_node(role, make_gemini_node(role, prompt))

for role in role_prompts:
    graph_builder.add_edge("Dispatch", role)

# Add Praetor and HumanInterrupt
graph_builder.add_node("Praetor", praetor_node)
graph_builder.add_node("HumanInterrupt", human_interrupt_node)
graph_builder.add_conditional_edges("HumanInterrupt", check_continue, {"yes": "Dispatch", "no": END})

# Wire flow
graph_builder.set_entry_point("Dispatch")
for role in role_prompts:
    graph_builder.add_edge(role, "Praetor")
graph_builder.add_edge("Praetor", "HumanInterrupt")


graph = graph_builder.compile()

In [ ]:
# Graph Visualization
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Visualization failed.")
    pass

In [ ]:
# run the graph

initial_state = {
    "proposal": question,
    "loop_count": 1,
    "evaluations": {},
    "praetor_outputs": {},
    "continue_": True
}

state = initial_state
while state["continue_"]:
    print(f"\n===== Loop {state['loop_count']} =====")
    state = graph.invoke(state)
    state["loop_count"] += 1
